# Necessary imports and Data Loading

In [74]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Loading Data

In [75]:
data_set_directory = '../input/10-monkey-species'
training_path = data_set_directory + '/training/training'
validation_path = data_set_directory + '/validation/validation'

In [76]:
image_size = (224, 224)
batch_size = 64

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    training_path,
    label_mode = 'int',
    seed = 1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    validation_path,
    label_mode = 'int',
    seed =1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 1097 files belonging to 10 classes.
Found 272 files belonging to 10 classes.


# Training data augmentation

In [77]:
# Data Augmentation Layer for training

data_augmentation_train = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
        layers.experimental.preprocessing.Rescaling(scale =1./255),
        layers.experimental.preprocessing.RandomHeight(0.1),
        layers.experimental.preprocessing.RandomWidth(0.1)
     
    ]
)

# Testing data augmentation

In [78]:
# Data Augmentation Layer for testing

data_augmentation_test = keras.Sequential(
    [
        layers.experimental.preprocessing.Rescaling(scale =1./255)
     
    ]
)

Mapping the augmentations into the dataset

In [79]:
# Map the Data now

augmented_train_ds = train_ds.map(
  lambda x, y: (data_augmentation_train(x, training=True), y))

augmented_val_ds = val_ds.map(
  lambda x, y: (data_augmentation_test(x, training=True), y))

# Training Using the Transfer learning
See below for the training using a custom deep neural architecture

In [80]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3), include_top=False, weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

In [81]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import regularizers

In [87]:
global_average_layers = tf.keras.layers.GlobalAveragePooling2D()
dropouts = tf.keras.layers.Dropout(rate = 0.2)

dense_128 = tf.keras.layers.Dense(128)
dense_64 = tf.keras.layers.Dense(64)
dense_32 = tf.keras.layers.Dense(32)

prediction_layer = tf.keras.layers.Dense(10,activation='softmax')

model_V3 = tf.keras.Sequential([
    pre_trained_model,
    global_average_layers,
    dropouts,
    dense_128,
    dense_64,
    dropouts,
    dense_32,
    prediction_layer
])

model_V3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [88]:
model_V3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3

In [89]:
# Callbacks
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

filepath = "custom_training_model.h5"

tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callback_list = [tensorboard_callback, lr_reduce, checkpoint]

In [90]:
tf.keras.backend.clear_session()

In [91]:
history = model_V3.fit(
           augmented_train_ds, 
           epochs=6, validation_data=augmented_val_ds, callbacks=callback_list)

Epoch 1/6
18/18 [==============================] - ETA: 0s - loss: 1.8802 - accuracy: 0.5861
Epoch 00001: val_loss improved from inf to 0.22962, saving model to custom_training_model.h5
18/18 [==============================] - 21s 1s/step - loss: 1.8802 - accuracy: 0.5861 - val_loss: 0.2296 - val_accuracy: 0.9412
Epoch 2/6
18/18 [==============================] - ETA: 0s - loss: 0.3825 - accuracy: 0.8888
Epoch 00002: val_loss improved from 0.22962 to 0.17638, saving model to custom_training_model.h5
18/18 [==============================] - 18s 1s/step - loss: 0.3825 - accuracy: 0.8888 - val_loss: 0.1764 - val_accuracy: 0.9596
Epoch 3/6
18/18 [==============================] - ETA: 0s - loss: 0.2412 - accuracy: 0.9253
Epoch 00003: val_loss improved from 0.17638 to 0.16558, saving model to custom_training_model.h5
18/18 [==============================] - 19s 1s/step - loss: 0.2412 - accuracy: 0.9253 - val_loss: 0.1656 - val_accuracy: 0.9669
Epoch 4/6
18/18 [==============================

In [92]:
try:
    del model_V3
except Exception as e:
    print(e)

In [93]:
model = keras.models.load_model('custom_training_model.h5')

In [94]:
import numpy as np
from keras.preprocessing import image
from PIL import Image
import requests
import io
import os

def prepare_from_url(link):
    bytesdata = requests.get(link).content
    Image.open(io.BytesIO(bytesdata)).save('hehe.jpg')
    
    img_width, img_height = 224, 224
    img = image.load_img('hehe.jpg', target_size = (img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    
    os.remove("hehe.jpg")

    return img
    
img = prepare_from_url('https://upload.wikimedia.org/wikipedia/commons/d/de/Silvery_Marmoset_%2810511829704%29.jpg')
pred = model.predict(img)

In [95]:
pred

array([[0.0000000e+00, 7.4124638e-29, 0.0000000e+00, 1.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 2.8951564e-20, 0.0000000e+00,
        5.7335915e-29, 0.0000000e+00]], dtype=float32)

In [96]:
labels_latin = ['alouatta_palliata', 
          'erythrocebus_patas', 
          'cacajao_calvus', 
          'macaca_fuscata', 
          'cebuella_pygmea', 
          'cebus_capucinus', 
          'mico_argentatus', 
          'saimiri_sciureus', 
          'aotus_nigriceps', 
          'trachypithecus_johnii']

labels_common = ['mantled_howler', 
                 'patas_monkey', 
                 'bald_uakari', 
                 'japanese_macaque',
                 'pygmy_marmoset',
                 'white_headed_capuchin',
                 'silvery_marmoset',
                 'common_squirrel_monkey',
                 'black_headed_night_monkey',
                 'nilgiri_langur']

print(f'Model has predicted the image to be an {labels_common[np.argmax(pred)]} whose latin name is {labels_latin[np.argmax(pred)]}')

Model has predicted the image to be an japanese_macaque whose latin name is macaca_fuscata


In [ ]:
!tensorboard dev upload --logdir log \
    --name "MonkeyNet for monkey classification" \
    --description "Made by theroyakash"

# Using MobileNet v2